### Evaluating various models on validation Videos

In [1]:
# Widjet to reload all imports as they are updated is:
%reload_ext autoreload
from ultralytics import YOLO
import pandas as pd
import wandb
import os
import sys
from pathlib import Path
from evaluate_yolo_tracker import evaluate_per_sequence

Error importing BURST due to missing underlying dependency: No module named 'pycocotools'


In [2]:
model_names = ['p3v6_v8n_1000e_augs_ns_backgrounds',
               'p3v6_v8s_1000e_v8s_augs_ns_backgrounds8',
               'p3v6_v8s_1000e_mosaic0.8_perspective0.0005_cutmix0.1',
               'p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15',
               'p3v6_v8s_1000e2',
               'p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1',
               ]
def load_model(model_version):
    project = '/vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/'
    model_folder_path = os.path.join(project, 'models', model_version)
    model_path = os.path.join(model_folder_path, 'weights/best.pt')
    return YOLO(model_path)

In [3]:
# Load current production SharkTrack model
model = YOLO("/vol/biomedic3/bglocker/ugproj/tk1420/sharktrack/models/sharktrack.pt")

In [ ]:
def find_confidence_for_target_fp(model: YOLO,
                                  target_fp: int,
                                  data_yaml: str='/vol/biomedic3/bglocker/ugproj/tk1420/p3v6/data_config.yaml',
                                  tol: int = 1,
                                  max_iter: int = 20,
                                  debug: bool = False) -> float:
    """
    Perform a binary search over confidence thresholds to find the threshold
    at which the number of false positives (FP) matches the target_fp within tol.

    Args:
        model: An initialized ultralytics YOLO model.
        data_yaml: Path to the data YAML file for validation.
        target_fp: Desired number of false positives.
        tol: Acceptable tolerance for FP difference (default: 1).
        max_iter: Maximum number of binary search iterations (default: 20).
        debug: If True, prints intermediate FP counts and thresholds.

    Returns:
        Best confidence threshold achieving FP within tol of target_fp.
    """
    low, high = 0.0, 1.0
    best_thr = None
    best_diff = float('inf')

    for i in range(max_iter):
        mid = (low + high) / 2
        # Run validation at this confidence threshold
        results = model.val(conf=mid, data=data_yaml, plots=True, save_json=False, verbose=False, iou=0.5)
        # Extract 2x2 confusion matrix: [[TN, FP], [FN, TP]]
        cm = results.confusion_matrix.matrix
        fp = int(cm[0][1])  # false positives

        diff = abs(fp - target_fp)
        if debug:
            print(f"Confusion Matrix at thr={mid:.4f}:\n{cm}")
            print(f"Iter {i}: thr={mid:.4f}, FP={fp}, diff={diff}")

        # Update best found
        if diff < best_diff:
            best_diff = diff
            best_thr = mid

        # If within tolerance, break
        if diff <= tol:
            break

        # Adjust search range:
        # If FP > target, need stricter (higher) threshold to reduce FP.
        if fp > target_fp:
            low = mid
        # If FP < target, need looser (lower) threshold to increase FP.
        else:
            high = mid

    if debug:
        print(f"Best thr={best_thr:.4f}, FP diff={best_diff}")
    
    return best_thr


In [13]:
# Get the optimal conf threshold for each model (to achieve FPs similar to production model)
TP, FP = 949, 146  # True Positives, False Positives from production model
modelToConf = {}

for model_name in model_names:
    model = load_model(model_name)
    threshold = find_confidence_for_target_fp(model, target_fp=FP, debug=True, tol=3)
    print(f"Model: {model_name}, Conf Threshold: {threshold:.4f}")
    modelToConf[model_name] = threshold
    
print("Confidence thresholds for models:")
for model_name, conf in modelToConf.items():
    print(f"{model_name}: {conf:.4f}")

Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:05<00:00, 11.61it/s]


                   all        980       1174      0.934      0.738      0.846      0.612
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/val285
Confusion Matrix at thr=0.5000:
[[        874          15]
 [        300           0]]
Iter 0: thr=0.5000, FP=15, diff=131
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:04<00:00, 14.96it/s]


                   all        980       1174      0.889      0.794      0.871      0.616
Speed: 0.2ms preprocess, 0.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/val286
Confusion Matrix at thr=0.2500:
[[        962          65]
 [        212           0]]
Iter 1: thr=0.2500, FP=65, diff=81
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)


...

                   all        980       1174      0.877      0.803      0.874      0.629
Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/val330
Confusion Matrix at thr=0.1094:
[[       1005         135]
 [        169           0]]
Iter 5: thr=0.1094, FP=135, diff=11
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:05<00:00, 10.46it/s]


                   all        980       1174      0.877      0.803      0.874      0.629
Speed: 0.3ms preprocess, 1.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/val331
Confusion Matrix at thr=0.1016:
[[       1010         139]
 [        164           0]]
Iter 6: thr=0.1016, FP=139, diff=7
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:05<00:00, 10.50it/s]


                   all        980       1174      0.877      0.803      0.876      0.629
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/val332
Confusion Matrix at thr=0.0977:
[[       1014         146]
 [        160           0]]
Iter 7: thr=0.0977, FP=146, diff=0
Best thr=0.0977, FP diff=0
Model: p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1, Conf Threshold: 0.0977
Confidence thresholds for models:
p3v6_v8n_1000e_augs_ns_backgrounds: 0.1133
p3v6_v8s_1000e_v8s_augs_ns_backgrounds8: 0.0664
p3v6_v8s_1000e_mosaic0.8_perspective0.0005_cutmix0.1: 0.1016
p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15: 0.1055
p3v6_v8s_1000e2: 0.0059
p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1: 0.0977


In [3]:
# modelToConf={'p3v6_v8n_1000e_augs_ns_backgrounds': 0.234375, 
#              'p3v6_v8s_1000e_v8s_augs_ns_backgrounds8': 0.1796875, 
#              'p3v6_v8s_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.2109375, 
#              'p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15': 0.25, 
#              'p3v6_v8s_1000e2': 0.052734375, 
#              'p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.25
#             }
# The above appears to not apply anymore...

modelToConf = {'p3v6_v8n_1000e_augs_ns_backgrounds': 0.11328125, 
               'p3v6_v8s_1000e_v8s_augs_ns_backgrounds8': 0.06640625, 
               'p3v6_v8s_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.1015625, 
               'p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15': 0.10546875, 
               'p3v6_v8s_1000e2': 0.005859375, 
               'p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.09765625}



In [4]:
print(modelToConf)

{'p3v6_v8n_1000e_augs_ns_backgrounds': 0.11328125, 'p3v6_v8s_1000e_v8s_augs_ns_backgrounds8': 0.06640625, 'p3v6_v8s_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.1015625, 'p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15': 0.10546875, 'p3v6_v8s_1000e2': 0.005859375, 'p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1': 0.09765625}


In [4]:
# Set params for each model to then log to wandb
modelToParams = {}
for model_name, conf in modelToConf.items():
    
    model_folder_path = os.path.join('/vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/models', model_name)
    yaml_path = os.path.join(model_folder_path, 'args.yaml')
    
    # inspect the model args.yaml to get the following params
    with open(yaml_path, 'r') as f:
        args = f.read()

    # Parse the args.yaml to extract necessary parameters
    args = dict(line.strip().split(': ') for line in args.splitlines() if ': ' in line)
    model_size = args.get('model', 'a_name_n.pt').split('.')[0][-1]
    
    modelToParams[model_name] = {
        'name': model_name,
        'model_size': model_size,
        'pretrained_model': args.get('model', 'null'),
        'epochs': args.get('epochs', 1000),
        'imgsz': 640,
        'patience': args.get('patience', 50),
        'data_yaml': args.get('data', '/vol/biomedic3/bglocker/ugproj/tk1420/p3v6/data_config.yaml'),
        'project_folder': 'models',
        'batch': args.get('batch', 64),
        'iou_association_threshold': 0.5,
        'tracker': 'botsort.yaml',
        'conf_threshold': conf,
        'yolo_version': 'yolov8',
        'augmentations': {
            'hsv_h': args.get('hsv_h', 0.015),
            'hsv_s': args.get('hsv_s', 0.7),
            'hsv_v': args.get('hsv_v', 0.4),
            'degrees': args.get('degrees', 0.0),
            'translate': args.get('translate', 0.1),
            'scale': args.get('scale', 0.5),
            'shear': args.get('shear', 0.0),
            'perspective': args.get('perspective', 0.0),
            'flipud': args.get('flipud', 0.0),
            'fliplr': args.get('fliplr', 0.5),
            'bgr': args.get('bgr', 0.0),
            'mosaic': args.get('mosaic', 1.0),
            'mixup': args.get('mixup', 0.0),
            'cutmix': args.get('cutmix', 0.0),
        }
    }

In [5]:
import wandb

In [5]:
data_yaml = '/vol/biomedic3/bglocker/ugproj/tk1420/p3v6/data_config.yaml'
project_folder = '/vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/'
# Now evaluate each model at its optimal confidence threshold and print confusion matrix

for model_name, conf_threshold in modelToConf.items():
    print(f"Evaluating model {model_name} at confidence threshold {conf_threshold:.4f}")
    model = load_model(model_name)
    
    job_name='eval1'
    
    # Evaluate the model on the SharkTrack dataset
    results = model.val(name=f"{job_name}_{model_name}_val_conf{conf_threshold:.4f}_",
                        data=data_yaml,
                        conf=conf_threshold,
                        plots=True,
                        save_json=False,
                        verbose=False,
                        iou=0.5)

    # Print confusion matrix
    cm = results.confusion_matrix.matrix
    print(f"Confusion Matrix for {model_name} at conf: {conf_threshold:.4f}")
    print(cm)
    
    # Evaluate per sequence metrics
    model_path = os.path.join(project_folder, 'models', model_name, 'weights', 'best.pt')
    metrics_per_sequence, track_time, device, all_aligned_annotations = evaluate_per_sequence(model_path, conf_threshold, iou_association_threshold=0.5,imgsz=640, tracker_type='tracker_5fps.yaml')
    print(f"Per-sequence metrics for {model_name}:")
    for seq, metrics in metrics_per_sequence.items():
        print(f"Sequence: {seq}, Metrics: {metrics}")
        
    # Log to wandb
    run_name = model_name + f"_conf{conf_threshold:.4f}_{job_name}"
    wandb.init(project="SharkTrack-Dev_v2", name=run_name, config=modelToParams[model_name], job_type=job_name)
    log_data = {}
    for seq, metrics in metrics_per_sequence.items():
        # Best map50
        log_data[f"{seq}/mota"] = metrics['MOTA']
        log_data[f"{seq}/motp"] = metrics['MOTP']
        log_data[f"{seq}/idf1"] = metrics['IDF1']
        
    log_data['track_time'] = track_time
    log_data['track_device'] = device
    log_data["all/map50"] = results.results_dict['metrics/mAP50(B)']
    log_data['all/mota'] = sum(metrics['MOTA'] for metrics in metrics_per_sequence.values()) / len(metrics_per_sequence)
    log_data['all/motp'] = sum(metrics['MOTP'] for metrics in metrics_per_sequence.values()) / len(metrics_per_sequence)
    log_data['all/idf1'] = sum(metrics['IDF1'] for metrics in metrics_per_sequence.values()) / len(metrics_per_sequence)
    wandb.log(log_data)
    wandb.finish()
    print(f"Logged results for {model_name} to wandb.")

Evaluating model p3v6_v8n_1000e_augs_ns_backgrounds at confidence threshold 0.1133
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:25<00:00,  2.46it/s]


                   all        980       1174      0.889      0.794      0.876      0.612
Speed: 0.4ms preprocess, 0.8ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/eval1_p3v6_v8n_1000e_augs_ns_backgrounds_val_conf0.1133_3
Confusion Matrix for p3v6_v8n_1000e_augs_ns_backgrounds at conf: 0.1133
[[        995         143]
 [        179           0]]
Evaluating easy1
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 6.17s
Evaluating easy2
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 6.56s
Evaluating medium1
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 7.07s
Evaluating medium2
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 6.07s
Evaluating difficult1
Running track mode..

wandb: Currently logged in as: tk1420 (tk1420-imperial-college-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


NorthSea1/idf1,▁
NorthSea1/mota,▁
NorthSea1/motp,▁
all/idf1,▁
all/map50,▁
all/mota,▁
all/motp,▁
difficult1/idf1,▁
difficult1/mota,▁
difficult1/motp,▁
difficult2/idf1,▁


...

Logged results for p3v6_v8s_1000e2 to wandb.
Evaluating model p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 at confidence threshold 0.0977
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/p3v6/val/labels.cache... 980 images, 242 backgrounds, 0 corrupt: 100%|██████████| 980/980 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [00:02<00:00, 20.75it/s]


                   all        980       1174      0.877      0.803      0.876      0.629
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/eval1_p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1_val_conf0.0977_2
Confusion Matrix for p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 at conf: 0.0977
[[       1014         146]
 [        160           0]]
Evaluating easy1
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 3.60s
Evaluating easy2
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 4.20s
Evaluating medium1
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 6.23s
Evaluating medium2
Running track mode...
Processing frame 201

Processed 201 frames in track mode.
Sequence processing time: 6.72s
Evalua

NorthSea1/idf1,▁
NorthSea1/mota,▁
NorthSea1/motp,▁
all/idf1,▁
all/map50,▁
all/mota,▁
all/motp,▁
difficult1/idf1,▁
difficult1/mota,▁
difficult1/motp,▁
difficult2/idf1,▁


Logged results for p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 to wandb.


In [35]:
print(results.results_dict['metrics/mAP50(B)'])

0.8664611982322571


### Evaluate the model on the the first NorthSea validation video to get mAp
```python

In [ ]:
ns_data_yaml = '/vol/biomedic3/bglocker/ugproj/tk1420/NorthSea1_val/data_config.yaml'

for model_name, conf_threshold in modelToConf.items():
    print(f"Evaluating model {model_name} on North Sea dataset at confidence threshold {conf_threshold:.4f}")
    model = load_model(model_name)
    
    job_name='ns1_eval'
    
    # Evaluate the model on the North Sea dataset
    results = model.val(name=f"{job_name}_{model_name}_val_conf{conf_threshold:.4f}_",
                        data=ns_data_yaml,
                        conf=conf_threshold,
                        plots=True,
                        save_json=False,
                        verbose=False,
                        iou=0.5)

    # Print confusion matrix
    cm = results.confusion_matrix.matrix
    print(f"Confusion Matrix for {model_name} at conf: {conf_threshold:.4f} on North Sea dataset")
    print(cm)
    
    print(f"Results for {model_name} on North Sea dataset: {results.results_dict['metrics/mAP50(B)']}")
    
    
    # log to wandb
    run_name = model_name + f"_conf{conf_threshold:.4f}_{job_name}"
    wandb.init(project="SharkTrack-Dev", name=run_name, config=modelToParams[model_name], job_type=job_name)
    log_data = {}
    log_data['map50_ns1'] = results.results_dict['metrics/mAP50(B)']
   

Evaluating model p3v6_v8n_1000e_augs_ns_backgrounds on North Sea dataset at confidence threshold 0.2344
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/NorthSea1_val/val/labels... 191 images, 309 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1099.05it/s]


val: New cache created: /vol/biomedic3/bglocker/ugproj/tk1420/NorthSea1_val/val/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:02<00:00, 13.45it/s]

                   all        500        191          0          0          0          0


...

                   all        500        191     0.0429     0.0262     0.0117    0.00325
Speed: 0.4ms preprocess, 0.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/ns1_eval_p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15_val_conf0.2500_
Confusion Matrix for p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15 at conf: 0.2500 on North Sea dataset
[[         25           7]
 [        166           0]]
Results for p3v6_v8n_1000e_mosaic0.8_perspective0.0005_cutmix0.15 on North Sea dataset: 0.011747632263596021
Evaluating model p3v6_v8s_1000e2 on North Sea dataset at confidence threshold 0.0527
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/NorthSea1_val/val/labels.cache... 191 images, 309 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:02<00:00, 12.68it/s]


                   all        500        191     0.0203     0.0262     0.0104    0.00187
Speed: 0.9ms preprocess, 1.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/ns1_eval_p3v6_v8s_1000e2_val_conf0.0527_
Confusion Matrix for p3v6_v8s_1000e2 at conf: 0.0527 on North Sea dataset
[[          8           0]
 [        183           0]]
Results for p3v6_v8s_1000e2 on North Sea dataset: 0.010448280638433442
Evaluating model p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 on North Sea dataset at confidence threshold 0.2500
Ultralytics 8.3.0 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16072MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /vol/biomedic3/bglocker/ugproj/tk1420/NorthSea1_val/val/labels.cache... 191 images, 309 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:03<00:00, 10.56it/s]


                   all        500        191     0.0399      0.126     0.0235    0.00656
Speed: 0.4ms preprocess, 1.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /vol/biomedic3/bglocker/ugproj/tk1420/SharkTrack-Dev/runs/detect/ns1_eval_p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1_val_conf0.2500_
Confusion Matrix for p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 at conf: 0.2500 on North Sea dataset
[[         36           0]
 [        155           0]]
Results for p3v6_v11_1000e_mosaic0.8_perspective0.0005_cutmix0.1 on North Sea dataset: 0.02350218656995743
